# Example 3 - Labels correction and multitemporal table

<img src="images/banner3.png" width="100%" />

<font face="Calibri">
<br>
<font size="5"> <b>Sand classification, beachface clipping and multitemporal analysis</b></font>

<br>
<font size="4"> <b> Nicolas Pucino; PhD Student @ Deakin University, Australia </b> <br>
<img style="padding:7px;" src="images/sandpiper_sand_retouched.png" width="170" align="right" /></font>

<font size="3">This notebook illustrates how to use assign the final Sand or no-sand labels to the points, clip only beachface areas and create an organised dataframe storing elevation changes from each period available in all locations. <br>

<b>This notebook covers the following concepts:</b>

- Sand vs No-Sand classification.
- Beachface clipping.
- Multitemporal extraction
</font>


</font>

In [1]:
import pandas as pd
import geopandas as gpd
from datetime import datetime
from tqdm.notebook import tqdm

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
labelled_dataset=pd.read_csv(r"C:\my_packages\doc_data\labels\data_classified.csv")

In [3]:
# QGIS Labels

In [4]:
# define dictionaries and lists

labels_dict={"water":0,
            "sand":1,
            "vegetation":2,
            "no_sand":3}

labels_sand=[1]
labels_no_sand=[0,2,3]


In [102]:
water_dict= {'mar_20180601': [],
 'inv_20201020': [],
 'inv_20200826': []}

no_sand_dict={'inv_20201211': [7],
 'inv_20201020': [],
 'inv_20200826': []}

veg_dict={'inv_20201211': [],
 'inv_20201020': [],
 'inv_20200826': []}

sand_dict={'mar_20180601: [3,5,8,9],
 'inv_20201020': [0,2,3,6,7,8],
 'inv_20200826': [1,3,4,7,8]}

## Reclassification of sand labels

In [213]:
%%time

# classify sand VS no-sand

df_labels=labelled_dataset

corrected_labelled_df=pd.DataFrame()
list_locs=labelled_dataset.location.unique()

for location in tqdm(list_locs):
    
    print(f"{location}.")
    
    list_dates = df_labels.query(f"location=='{location}'").raw_date.unique()
    
    for survey_date in list_dates:
                        
        dataset_str=f"{location}_{survey_date}"
        print(f"Working on: {dataset_str}.")
        data_in=df_labels.query(f"location=='{location}' & raw_date=='{survey_date}'")
        

        list_labels=data_in.label_k.unique()   

        # water
        try:
            print(f"{dataset_str}, evaluating WATER.")
            label_df=data_in.query(f"label_k == {water_dict[dataset_str]} ")
            label_df["opt_label"]=labels_dict["water"]
            corrected_labelled_df=pd.concat([label_df,corrected_labelled_df], ignore_index=True)
            list_labels=[e for e in list_labels if e not in water_dict[dataset_str]]
        
        except:
            print(f"{dataset_str} does not have water classes")

        # sand
        try:
            print(f"{dataset_str}, evaluating SAND.")
            label_df=data_in.query(f"label_k == {sand_dict[dataset_str]} ")
            label_df["opt_label"]=labels_dict["sand"]
            corrected_labelled_df=pd.concat([label_df,corrected_labelled_df], ignore_index=True)
            list_labels=[e for e in list_labels if e not in sand_dict[dataset_str]]

        except:
            print(f"{dataset_str} does not have sand classes")

        # veg
        try:
            print(f"{dataset_str}, evaluating VEG.")
            label_df=data_in.query(f"label_k == {veg_dict[dataset_str]} ")
            label_df["opt_label"]=labels_dict["vegetation"]
            corrected_labelled_df=pd.concat([label_df,corrected_labelled_df], ignore_index=True)
            list_labels=[e for e in list_labels if e not in veg_dict[dataset_str]]
            
        except:
            print(f"{dataset_str} does not have vegetation classes")

        # no_sand
        try:
            print(f"{dataset_str}, evaluating NO_SAND.")
            label_df=data_in.query(f"label_k == {no_sand_dict[dataset_str]} ")
            label_df["opt_label"]=labels_dict["no_sand"]
            corrected_labelled_df=pd.concat([label_df,corrected_labelled_df], ignore_index=True)
            list_labels=[e for e in list_labels if e not in no_sand_dict[dataset_str]]
        except:
            print(f"{dataset_str} does not have no_sand classes")
            
            
        
        print(f"In {dataset_str}, the remaining labels {list_labels} are classified as no_sand.")
            
        label_df=data_in.query(f"label_k == {list_labels} ")
        label_df["opt_label"]=labels_dict["no_sand"]            
        corrected_labelled_df=pd.concat([label_df,corrected_labelled_df], ignore_index=True)

print("Checking for duplicated rows . . . ")

if corrected_labelled_df.point_id.is_unique == False:
    
    print("There are some duplicated labels in the dictioanries. Run "'duplicated_df.groupby(by=["location","survey_date","label_k"]).count()'" to find out. PLease correct them and re-run the cell. ")
    # Select duplicate rows 
    duplicated_df = corrected_labelled_df[corrected_labelled_df.duplicated(['point_id'])]
    duplicated_df.groupby(by=["location","survey_date","label_k"]).count()
else:
    
    print("Reclassification run successfully!")


corrected_labelled_df['sand_label'] = [ 1 if s in labels_no_sand else 0 for s in corrected_labelled_df.opt_label.values]
corrected_labelled_df.sand_label.value_counts()


inv.
Working on: inv_20201211.
inv_20201211, evaluating WATER.
inv_20201211, evaluating SAND.


C:\Users\npucino\AppData\Local\Continuum\anaconda3\envs\transectenvi\lib\site-packages\ipykernel\__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


inv_20201211, evaluating VEG.
inv_20201211, evaluating NO_SAND.


C:\Users\npucino\AppData\Local\Continuum\anaconda3\envs\transectenvi\lib\site-packages\ipykernel\__main__.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In inv_20201211, the remaining labels [1, 4, 2, 6] are classified as no_sand.


C:\Users\npucino\AppData\Local\Continuum\anaconda3\envs\transectenvi\lib\site-packages\ipykernel\__main__.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Working on: inv_20201020.
inv_20201020, evaluating WATER.
inv_20201020, evaluating SAND.
inv_20201020, evaluating VEG.
inv_20201020, evaluating NO_SAND.
In inv_20201020, the remaining labels [9, 5, 4, 1] are classified as no_sand.
Working on: inv_20200826.
inv_20200826, evaluating WATER.
inv_20200826, evaluating SAND.
inv_20200826, evaluating VEG.
inv_20200826, evaluating NO_SAND.
In inv_20200826, the remaining labels [6, 0, 9, 5, 2] are classified as no_sand.

Checking for duplicated rows . . . 
Reclassification run successfully!
Wall time: 5.43 s


In [215]:
# get a geodataframe with point objects

corrected_labelled_df = corrected_labelled_df.loc[:,~corrected_labelled_df.columns.duplicated()] # eliminate duplicated columns
corrected_labelled_df['geometry']=corrected_labelled_df.coordinates.apply(coords_to_points) # create Point objects
corrected_labelled_gdf=gpd.GeoDataFrame(corrected_labelled_df, geometry='geometry', crs=crs_dict_string['inv']) # create GeoDataFrame
corrected_labelled_gdf

,distance,z,tr_id,raw_date,coordinates,location,survey_date,point_id,x,y,geometry,band1,band2,band3,slope,curve,label_k,opt_label,sand_label
0,0.0,-0.416860,163,20200826,POINT (389485.0292927367 5722796.792901353),inv,2020-08-26,2601v22837030600in006,389485.0292927367,5722796.792901353,POINT (389485.029 5722796.793),94.0,91.0,95.0,-2.766547,1.376038,6,3,1
1,0.1,-0.419285,163,20200826,POINT (389485.0122613082 5722796.891440332),inv,2020-08-26,2601v22832000810in006,389485.0122613082,5722796.891440332,POINT (389485.012 5722796.891),93.0,89.0,93.0,-0.002408,1.382086,6,3,1
2,0.2,-0.421676,163,20200826,POINT (389484.9952298797 5722796.989979312),inv,2020-08-26,2601v22837070920in006,389484.9952298797,5722796.989979312,POINT (389484.995 5722796.990),89.0,85.0,89.0,-0.002375,0.000033,0,3,1
3,0.3,-0.424035,163,20200826,POINT (389484.9781984512 5722797.088518291),inv,2020-08-26,2601v22832050130in006,389484.97819845116,5722797.088518291,POINT (389484.978 5722797.089),85.0,83.0,86.0,-0.002343,-0.000652,0,3,1
4,0.4,-0.426361,163,20200826,POINT (389484.9611670226 5722797.18705727),inv,2020-08-26,2601v22836020240in006,389484.96116702264,5722797.18705727,POINT (389484.961 5722797.187),88.0,85.0,91.0,-0.003679,-0.000640,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320347,92.5,3.377417,0,20201211,POINT (386871.7613396471 5721639.905428521),inv,2020-12-11,10204001i2272101nv95,386871.7613396471,5721639.905428521,POINT (386871.761 5721639.905),97.0,116.0,70.0,0.012603,0.006434,9,1,0
320348,92.6,3.402572,0,20201211,POINT (386871.7460518774 5721640.004253033),inv,2020-12-11,10207001i2272401nv96,386871.7460518774,5721640.004253033,POINT (386871.746 5721640.004),86.0,112.0,61.0,0.012578,-0.006844,9,1,0
320349,92.7,3.402572,0,20201211,POINT (386871.7307641076 5721640.103077544),inv,2020-12-11,10200001i2272601nv97,386871.7307641076,5721640.103077544,POINT (386871.731 5721640.103),101.0,128.0,76.0,-0.001086,-0.006841,9,1,0
320350,92.8,3.400400,0,20201211,POINT (386871.7154763379 5721640.201902056),inv,2020-12-11,10203001i2272901nv98,386871.7154763379,5721640.201902056,POINT (386871.715 5721640.202),103.0,127.0,76.0,-0.001105,-0.006097,9,1,0


## Polygon correction

In [10]:
%%time
# apply poly correction with geopandas

#labelled_gdf=corrected_labelled_gdf
corr_date_field='survey_date'
labelled_df_date_field='raw_date'

is_sand_path=r"E:\chapter_4\chloe_inv\add3\labels\chloe_add3_ToSand.gpkg"
is_not_sand_path=r"E:\chapter_4\chloe_inv\add3\labels\chloe_add3_Not_Sand.gpkg"
shore_path=r"C:\jupyter\shore_areas\inv_shore.gpkg"

#____________
chloe_vali_noSand=gpd.read_file(is_not_sand_path)

if '-' in chloe_vali_noSand.loc[:,corr_date_field].any():
    chloe_vali_noSand.loc[:,corr_date_field]=chloe_vali_noSand.loc[:,corr_date_field].apply(lambda x: x.replace('-',''))
else:
    pass

if corr_date_field != 'date_raw':
    chloe_vali_noSand.rename({corr_date_field:'date_raw'}, axis=1, inplace=True)
else:
    pass

chloe_vali_Sand=gpd.read_file(is_sand_path)

if '-' in chloe_vali_Sand.loc[:,corr_date_field].any():
    chloe_vali_Sand.loc[:,corr_date_field]=chloe_vali_Sand.loc[:,corr_date_field].apply(lambda x: x.replace('-',''))
else:
    pass

if corr_date_field != 'date_raw':
    chloe_vali_Sand.rename({corr_date_field:'date_raw'}, axis=1, inplace=True)
else:
    pass


# check wether the correction geopackages are empty or not. If empty, skip correction.
if chloe_vali_Sand.empty:
    skip_isSand=True
    print("Correction TO SAND skipped as the provided file is empty.")
else:
    skip_isSand=False
    
if chloe_vali_noSand.empty:
    skip_NoSand=True
    print("Correction TO NO-SAND skipped as the provided file is empty.")
else:
    skip_NoSand=False
    

# Poly correction starts here______________________________________

to_update=pd.DataFrame()

for date_in in labelled_gdf.loc[:,labelled_df_date_field].unique():

    #subset points and polygones based on date
    data_in=labelled_gdf.query(f"{labelled_df_date_field} == '{date_in}'")
    vali_isSand=chloe_vali_Sand.query(f"date_raw == '{date_in}'")
    vali_NoSand=chloe_vali_noSand.query(f"date_raw == '{date_in}'")
    
    # first set what IS SAND
    if bool(skip_isSand)==True or vali_isSand.empty:
        pass
    else:
        for i in range(vali_isSand.shape[0]): # loops through all the polygones

            target_k=int(vali_isSand.iloc[i]['target_label_k'])

            if target_k != 999:

                data_in=labelled_gdf.query(f"{labelled_df_date_field} == '{date_in}' & label_k=='{target_k}'")
                selection=data_in[data_in.geometry.intersects(vali_isSand.geometry.iloc[i])]
                selection["corr_label"]=0

            elif target_k == 999:

                data_in=labelled_gdf.query(f"{labelled_df_date_field} == '{date_in}'")
                selection=data_in[data_in.geometry.intersects(vali_isSand.geometry.iloc[i])]
                selection["corr_label"]=0

        to_update=pd.concat([selection,to_update], ignore_index=True)
        
            
    # Now set what IS NOT SAND
    if skip_NoSand or vali_NoSand.empty:
        pass
    else:
        for i in range(vali_NoSand.shape[0]): # loops through all the polygones

            target_k=int(vali_NoSand.iloc[i]['target_label_k'])

            if target_k != 999:

                data_in=labelled_gdf.query(f"{labelled_df_date_field} == '{date_in}' & label_k=='{target_k}'")
                selection=data_in[data_in.geometry.intersects(vali_NoSand.geometry.iloc[i])]
                selection["corr_label"]=1

            elif target_k == 999:

                data_in=labelled_gdf.query(f"{labelled_df_date_field} == '{date_in}'")
                selection=data_in[data_in.geometry.intersects(vali_NoSand.geometry.iloc[i])]
                selection["corr_label"]=1

        to_update=pd.concat([selection,to_update], ignore_index=True)
    
to_update.drop_duplicates(subset="point_id", inplace=True)


#__CREATE NEW UPDATED DATAFRAME____________________________
labelled_df_updated=pd.merge(left=labelled_gdf, right=to_update.loc[:,['point_id','corr_label']], # Left Join 
                             how='left', validate='one_to_one') 
labelled_df_updated.corr_label.fillna(labelled_df_updated.sand_label, inplace=True) # Fill NaN with previous sand labels
labelled_df_updated["corr_label"]=labelled_df_updated.corr_label.astype(int) # Transform corr_labels in Int

#__CLIP BY SHORE____________________________
shore=gpd.read_file(shore_path)
in_shore=labelled_df_updated[labelled_df_updated.geometry.intersects(shore.geometry.iloc[0])]


print("Done")
in_shore.head()

DriverError: E:\chapter_4\chloe_inv\add3\labels\chloe_add3_Not_Sand.gpkg: No such file or directory

In [284]:
in_shore.to_csv(r"E:\chapter_4\chloe_inv\add3\labels\add3_inv_labelled_inshore.csv", index=False)

## Create multitemporal datased (dh)

In [3]:
## Multitemporal Extraction to loop trough locations

full_dataset=pd.read_csv(r"C:\my_packages\doc_data\profiles\classified_data.csv")

In [29]:
def compute_multitemporal (df,
                           filter_sand=True,
                           date_field='survey_date',
                          sand_label_field='label_sand',
                           filter_classes=[0],
                          common_field="geometry"):
    """
    From a dataframe containing the extracted points and a column specifying wether they are sand or non-sand, returns a multitemporal dataframe
    with time-periods sand-specific elevation changes.

    Args:
        date_field (str): the name of the column storing the survey date.
        sand_label_field (str): the name of the column storing the sand label (usually sand=0, no_sand=1).
        filter_classes (list): list of integers specifiying the label numbers of the sand_label_field that are sand. Default [0].
        common_field (str): name of the field where the points share the same name. It is usually the geometry or spatial IDs.

    Returns:
        A multitemporal dataframe of sand-specific elevation changes.
    """


    fusion_long=pd.DataFrame()

    for location in df.location.unique():
        print(f"working on {location}")
        loc_data=df.query(f"location=='{location}'")
        list_dates=loc_data.loc[:,date_field].unique()
        list_dates.sort()


        for i in tqdm(range(list_dates.shape[0])):

            if i < list_dates.shape[0]-1:
                date_pre=list_dates[i]
                date_post=list_dates[i+1]
                print(f"Calculating dt{i}, from {date_pre} to {date_post} in {location}.")
                
                if filter_sand:
                    df_pre=loc_data.query(f"{date_field} =='{date_pre}' & {sand_label_field} in {filter_classes}").dropna(subset=['z'])
                    df_post=loc_data.query(f"{date_field} =='{date_post}' & {sand_label_field} in {filter_classes}").dropna(subset=['z'])
                else:
                    df_pre=loc_data.query(f"{date_field} =='{date_pre}'").dropna(subset=['z'])
                    df_post=loc_data.query(f"{date_field} =='{date_post}'").dropna(subset=['z'])
                
                merged=pd.merge(df_pre,df_post, how='inner', on=common_field,validate="one_to_one",suffixes=('_pre','_post'))
                merged["dh"]=merged.z_post.astype(float) - merged.z_pre.astype(float)

                dict_short={"geometry": merged.geometry,
                            "location":location,
                            "tr_id":merged.tr_id_pre,
                            "distance":merged.distance_pre,
                            "dt":  f"dt_{i}",
                            "date_pre":date_pre,
                            "date_post":date_post,
                            "z_pre":merged.z_pre.astype(float),
                            "z_post":merged.z_post.astype(float),
                            "dh":merged.dh}

                short_df=pd.DataFrame(dict_short)
                fusion_long=pd.concat([short_df,fusion_long],ignore_index=True)

    print("done")
    return fusion_long

In [30]:
full_dataset

,Unnamed: 0,point_id,location,survey_date,z,tr_id,distance,label_sand,spatial_id,geometry
0,0,62122091m2510340ar20,mar,2019-05-16,NaN,22,2.4,1,22ar0m24,POINT (731624.209562121 5705492.44046626)
1,1,62126091m2590390ar20,mar,2019-05-16,NaN,22,2.9,1,22ar0m29,POINT (731623.785562969 5705492.70546573)
2,2,64121091m2580400ar00,mar,2019-05-16,-0.004194,24,0.0,1,24ar0m00,POINT (731646.903760184 5705523.4689886)
3,3,64121091m2510250ar00,mar,2019-05-16,-0.004196,24,0.5,1,24ar0m05,POINT (731646.491030611 5705523.75121953)
4,4,64128091m2520270ar00,mar,2019-05-16,-0.004196,24,0.7,1,24ar0m07,POINT (731646.325938782 5705523.8641119)
...,...,...,...,...,...,...,...,...,...,...
105032,34455,ol906089201e002306,leo,2018-09-20,1.429813,69,30.6,1,0le6o6930,POINT (299898.0365287552 5773691.407968608)
105033,34456,ol92101208500300e,leo,2018-09-20,-0.244038,53,0.1,1,53eo0l01,POINT (300001.0901262025 5773392.246766388)
105034,34457,ol950820143e032010,leo,2018-09-20,1.162140,45,33.1,1,0le4o1533,POINT (300031.9095018596 5773232.738285414)
105035,34458,ole120903080295600,leo,2018-09-20,5.996941,50,39.6,1,0le5o6039,POINT (299987.0210992148 5773321.983921126)


In [27]:
dh_df=compute_multitemporal(full_dataset,
                      date_field='survey_date',
                      sand_label_field='label_sand')


working on mar


  0%|          | 0/9 [00:00<?, ?it/s]

Calculating dt0, from 2018-06-01 to 2018-06-21 in mar.
Calculating dt1, from 2018-06-21 to 2018-07-27 in mar.
Calculating dt2, from 2018-07-27 to 2018-09-25 in mar.
Calculating dt3, from 2018-09-25 to 2018-11-13 in mar.
Calculating dt4, from 2018-11-13 to 2018-12-11 in mar.
Calculating dt5, from 2018-12-11 to 2019-02-05 in mar.
Calculating dt6, from 2019-02-05 to 2019-03-13 in mar.
Calculating dt7, from 2019-03-13 to 2019-05-16 in mar.
working on leo


  0%|          | 0/7 [00:00<?, ?it/s]

Calculating dt0, from 2018-06-06 to 2018-07-13 in leo.
Calculating dt1, from 2018-07-13 to 2018-07-25 in leo.
Calculating dt2, from 2018-07-25 to 2018-09-20 in leo.
Calculating dt3, from 2018-09-20 to 2019-02-11 in leo.
Calculating dt4, from 2019-02-11 to 2019-03-28 in leo.
Calculating dt5, from 2019-03-28 to 2019-07-31 in leo.
done


In [28]:
dh_df

,geometry,location,tr_id,distance,dt,date_pre,date_post,z_pre,z_post,dh
0,POINT (299901.7782793006 5773692.070767866),leo,69,26.8,dt_5,2019-03-28,2019-07-31,0.223866,0.112838,-0.111029
1,POINT (299901.4828779417 5773692.018441608),leo,69,27.1,dt_5,2019-03-28,2019-07-31,0.256890,0.152005,-0.104885
2,POINT (299901.2859437025 5773691.983557437),leo,69,27.3,dt_5,2019-03-28,2019-07-31,0.283333,0.176710,-0.106624
3,POINT (299901.1874765829 5773691.966115351),leo,69,27.4,dt_5,2019-03-28,2019-07-31,0.305573,0.188821,-0.116752
4,POINT (299901.0890094633 5773691.948673265),leo,69,27.5,dt_5,2019-03-28,2019-07-31,0.329158,0.197999,-0.131160
...,...,...,...,...,...,...,...,...,...,...
38676,POINT (731460.3631030347 5705157.200825883),mar,3,52.4,dt_0,2018-06-01,2018-06-21,4.663569,4.665780,0.002210
38677,POINT (731460.263679132 5705157.211544445),mar,3,52.5,dt_0,2018-06-01,2018-06-21,4.780209,4.787802,0.007593
38678,POINT (731460.1642552292 5705157.222263007),mar,3,52.6,dt_0,2018-06-01,2018-06-21,4.875618,4.903659,0.028041
38679,POINT (731460.0648313265 5705157.232981569),mar,3,52.7,dt_0,2018-06-01,2018-06-21,4.970114,4.974202,0.004088


In [70]:
dh_df.to_csv(r"C:\my_packages\doc_data\profiles\dh_data.csv", index=False)

## Create Dataframe of details


In [32]:
create_details_df(dh_df, loc_full)

,dt,date_pre,date_post,location,n_days,loc_full
0,dt_0,2018-06-01,2018-06-21,mar,20,Marengo
1,dt_1,2018-06-21,2018-07-27,mar,36,Marengo
2,dt_2,2018-07-27,2018-09-25,mar,60,Marengo
3,dt_3,2018-09-25,2018-11-13,mar,49,Marengo
4,dt_4,2018-11-13,2018-12-11,mar,28,Marengo
5,dt_5,2018-12-11,2019-02-05,mar,56,Marengo
6,dt_6,2019-02-05,2019-03-13,mar,36,Marengo
7,dt_7,2019-03-13,2019-05-16,mar,64,Marengo
8,dt_0,2018-06-06,2018-07-13,leo,37,St. Leonards
9,dt_1,2018-07-13,2018-07-25,leo,12,St. Leonards


In [25]:
locs_dt_str.to_csv(r"C:\my_packages\doc_data\profiles\dt_info.csv", index=False)